# Github web crawler + API test


In [1]:
import pandas as pd;
import requests
import unittest
from bs4 import BeautifulSoup
#from multiprocessing import Pool
from ipywidgets import widgets
from IPython.display import display


In [2]:
website_main = "https://gist.github.com/paulmillr/2657075"
website_prefix = "https://api.github.com"
api_headers = {'Accept': 'application/vnd.github.v3+json'}

In [3]:
def _handle_request_result_and_build_soup(request_result):
    if request_result.status_code == 200:
        html_doc =  request_result.text
        soup = BeautifulSoup(html_doc,"html.parser")
        return soup


In [4]:
def _convert_string_to_float(string):
    temp = string.strip()
    if temp[0]== '(' : # small negative percentage
        return float(temp[1:-2])
    if temp[-1]== '-' : # -- NA value
        return 0.0
    temp = ''.join(temp.split(',')) # remove thousand's comma
    return float(temp)


In [5]:
def get_page_soup(page):
    res = requests.get(page)
    #r = requests.get(url, headers=api_headers)
    soup = _handle_request_result_and_build_soup(res)
    return soup


In [6]:
def get_page_api(page):
    res = requests.get(website_prefix + page, headers=api_headers, 
                       auth=(login, password))
    if res.status_code != 200:
        print("Git error: status code", res.status_code)
    return res.json()


In [7]:
def process_github(nb):
    s = get_page_soup(website_main)
    liste = s.find_all("tr", limit = (nb+1))
    liste_heads = liste[0].find_all("th", limit = 4)
    #liste_heads[0].text = #, titres des rangs
    #liste_heads[1].text = User
    #liste_heads[2].text = Contribs
    #liste_heads[3].text = Location
    #liste_heads[4].text = Picture, discard
    df = pd.DataFrame(columns=[liste_heads[1].text, liste_heads[2].text,
                           liste_heads[3].text, 'moyenne'])
    for i in range(1, len(liste)):
        if i%8 == 0:
            print('.', end='', flush=True)
        liste_user = liste[i].find_all("td", limit = 3) #premier utilisateur, rank
        #liste_user[0].a.text #name
        #liste_user[1].text #contribs
        #liste_user[2].text #location
        name = liste_user[0].a.text
        #print(i, ' ', name)
        #GET /users/:username/repos #Default: owner
        g = get_page_api("/users/" + name + "/repos")
        if len(g) > 0:
            sum = 0;
            for k in range(len(g)):
                #print(g[k]['name'] + ' ' + str(g[k]['stargazers_count']))
                sum += g[k]['stargazers_count']
            sum /= len(g)
        else:
            sum = 0 #at 179 thejameskyle !
        #print(sum)
        df.loc[str(i)] = pd.Series({liste_heads[1].text:name,
                                    liste_heads[2].text:liste_user[1].text,
                                    liste_heads[3].text:liste_user[2].text,
                                   'moyenne': sum})
    
    print("");
    return df;


In [8]:
login = "bgtel";
password = "blank"

text1 = widgets.Text(
    placeholder='login',
    description='Login:'
)

text2 = widgets.Text(
    placeholder='access token or password',
    description='Password:'
)

def handle_submit1(sender):
    global login #2 lignes requis
    login = text1.value
    print("Login stored")

def handle_submit2(sender):
    global password
    password = text2.value
    text2.value=u'*'*len(text2.value)
    print("Password stored")

text1.on_submit(handle_submit1)
text2.on_submit(handle_submit2)

print("Warning, the password my be sent in clear text in the header!")
print("Enter login infos here:")
display(text1)
display(text2)


Warning, the password my be sent in clear text in the header!
Enter login infos here:


Text(value='', description='Login:', placeholder='login')

Text(value='', description='Password:', placeholder='access token or password')

Password stored


# Lancer ici une fois les droits d'acc&egrave;s remplis ci-dessus

In [11]:
df = pd.DataFrame();
if password != "blank":
    nb = 256
    print("Progress to ------------------->")
    df = process_github(nb);
    df = df.sort_values(['moyenne'], ascending = [0])
else:
    print("Need login infos in the fields above!")
df.head(10)

Progress to -------------------->
................................


,User,Contribs,Location,moyenne
226,bailicangdu,1500,"Shanghai, China",3489.923077
19,kamranahmedse,4994,Dubai,3295.833333
143,vinta,2213,"Taipei, Taiwan",1946.933333
118,mrdoob,2555,,1856.366667
45,sebastianbergmann,4390,"Siegburg, Germany",1611.200000


In [13]:
# Pour verif d'acces
get_page_api("/rate_limit") #60 per hour sans user id, not enough

{'rate': {'limit': 5000, 'remaining': 3617, 'reset': 1539632768},
 'resources': {'core': {'limit': 5000, 'remaining': 3617, 'reset': 1539632768},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1539634452},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1539630912}}}